# Using MKL and Eigen

**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Setup and settings](#toc2_)    
- 3. [Compile C++](#toc3_)    
- 4. [Matrix multiplication](#toc4_)    
- 5. [Solve linear equation system](#toc5_)    
- 6. [Sparse matrices](#toc6_)    
- 7. [Clean up](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

This notebooks shows how to speed up matrix multiplication and solving (sparse) linear equation systems with the **Math Kernal Library (MKL)** from Intel and **[Eigen](https://eigen.tuxfamily.org/index.php?title=Main_Page)**. Using explicit parallization with OpenMP in particular provide a significant speed-up.

**Timing computer:** Windows 10 computer with 2x Intel(R) Xeon(R) Gold 6254 3.10 GHz CPUs (18 cores, 36 logical processes each) and 768 GB of RAM.

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import urllib.request
import numpy as np
import scipy.linalg
import scipy.sparse
import scipy.sparse.linalg
import scipy.io

import time

from EconModel import cpptools

## 2. <a id='toc2_'></a>[Setup and settings](#toc0_)

In [2]:
cpptools.setup_Eigen(do_print=True)

Eigen succesfully installed


**Settings:**

In [3]:
threads_vec = [(1,8),(2,4),(4,4)]

## 3. <a id='toc3_'></a>[Compile C++](#toc0_)

In [4]:
flags = '/LD /EHsc /O3 /openmp /std:c++17 /Qmkl /Icppfuncs/'
#flags = '/LD /EHsc /O3 /openmp /std:c++17 /Qmkl /QxCOMMON-AVX512 /Icppfuncs/'
example_MKL = cpptools.link_to_cpp('cppfuncs/example_MKL.cpp',
                                   options={'compiler':'intel','flags':flags})

## 4. <a id='toc4_'></a>[Matrix multiplication](#toc0_)

$$
C = \alpha A B + \beta C
$$

In [5]:
alpha = 1.0
beta = 0.0
    
def allocate_mat(n=300,nrep=500):
    
    # varying sizes to test code
    
    np.random.seed(17)
    
    A = np.random.uniform(size=(nrep,2*n,n))
    B = np.random.uniform(size=(nrep,n,3*n))
    C = np.random.uniform(size=(nrep,2*n,3*n))
    
    return A,B,C


In [6]:
def matmul_np(alpha,beta,A,B,C):
        
    t0 = time.perf_counter()
    C[:,:,:] = alpha*(A@B)+beta*C
    secs = time.perf_counter()-t0
    
    return secs


In [7]:
def matmul_BLAS(alpha,beta,A,B,C,threads_internal,threads_omp):
    
    nrep,m,k = A.shape
    nrep,k,n = B.shape
    nrep,m,n = C.shape
    
    secs = example_MKL.matmul_BLAS(nrep,m,n,k,alpha,beta,A,B,C,threads_internal,threads_omp)
    
    return secs
    

In [8]:
def matmul_Eigen(alpha,beta,A,B,C,threads_internal,threads_omp):
    
    nrep,m,k = A.shape
    nrep,k,n = B.shape
    nrep,m,n = C.shape
    
    secs = example_MKL.matmul_Eigen(nrep,m,n,k,alpha,beta,A,B,C,threads_internal,threads_omp)
    
    return secs
    

**Test with single matrix multiplication:**

In [9]:
threads = 4
for n in [500,1000,2000]:
    
    print(f'{n = }:') 
    
    # numpy
    A,B,C = allocate_mat(n,1)
    name = 'numpy:'
    secs = matmul_np(alpha,beta,A,B,C)
    print(f'{name:30s} {secs*1000:8.0f} ms')

    # mkl
    A,B,C = allocate_mat(n,1)
    name = f'mkl (no threads, no omp):'
    secs = matmul_BLAS(alpha,beta,A,B,C,1,1)
    print(f'{name:30s} {secs*1000:8.0f} ms')

    A,B,C = allocate_mat(n,1)
    name = f'mkl (all internal, no omp):'
    secs = matmul_BLAS(alpha,beta,A,B,C,threads,1)
    print(f'{name:30s} {secs*1000:8.0f} ms')
    
    # Eigen
    A,B,C = allocate_mat(n,1)
    name = f'Eigen (no threads, no omp):'
    secs = matmul_Eigen(alpha,beta,A,B,C,1,1)
    print(f'{name:30s} {secs*1000:8.0f} ms')

    A,B,C = allocate_mat(n,1)
    name = f'Eigen (all internal, no omp):'
    secs = matmul_Eigen(alpha,beta,A,B,C,threads,1)
    print(f'{name:30s} {secs*1000:8.0f} ms')    
    
    print('')

n = 500:
numpy:                               80 ms


mkl (no threads, no omp):            96 ms
mkl (all internal, no omp):          44 ms


Eigen (no threads, no omp):         133 ms
Eigen (all internal, no omp):        42 ms

n = 1000:


numpy:                              466 ms


mkl (no threads, no omp):           688 ms


...

numpy:                             1686 ms


mkl (no threads, no omp):          3220 ms


mkl (all internal, no omp):        1244 ms


Eigen (no threads, no omp):        3393 ms


Eigen (all internal, no omp):      1938 ms



**Test with multiple matrix multiplications:**

In [10]:
for (n,nrep) in [(200,400),(400,200)]:
    
    print(f'{n = }, {nrep = }:') 
    
    A,B,C = allocate_mat(n,nrep)
    
    # numpy
    name = 'numpy:'
    secs = matmul_np(alpha,beta,A,B,C)
    print(f'{name:30s} {secs*1000:8.0f} ms')
    C_np = C.copy()
    
    # mkl
    for (threads_internal,threads_omp) in threads_vec:
        
        name = f'mkl (internal={threads_internal:2d},omp={threads_omp:2d}):'
        C[:] = 0
        secs = matmul_BLAS(alpha,beta,A,B,C,threads_internal,threads_omp)
        print(f'{name:30s} {secs*1000:8.0f} ms')
        
    # check results are the same
    max_abs_diff = np.max(np.abs(C_np-C))
    assert np.isclose(max_abs_diff,0.0)   
    
    print('')
    

n = 200, nrep = 400:


numpy:                             2275 ms


mkl (internal= 1,omp= 8):           403 ms


mkl (internal= 2,omp= 4):           622 ms


mkl (internal= 4,omp= 4):           478 ms



n = 400, nrep = 200:


numpy:                            14858 ms


mkl (internal= 1,omp= 8):          2069 ms


mkl (internal= 2,omp= 4):          2069 ms


mkl (internal= 4,omp= 4):          2203 ms


## 5. <a id='toc5_'></a>[Solve linear equation system](#toc0_)

$$
Ax = b
$$

In [11]:
def allocate_solve_mat(n=500,nrhs=100,nrep=500):
    
    np.random.seed(17)

    A = np.random.uniform(size=(nrep,n,n))
    x = np.random.uniform(size=(nrep,n,nrhs))
    b = A@x

    return A,b,x

In [12]:
def solve_np(A,b):
    
    t0 = time.perf_counter()
    x = np.zeros(b.shape)
    for i in range(A.shape[0]):
        x[i,:,:] = np.linalg.solve(A[i],b[i])
    secs = time.perf_counter()-t0
    
    return x,secs

In [13]:
def solve_LAPACKE(A,b,x,threads_internal,threads_omp):
    
    nrep,n,_ = A.shape
    _,_,nrh = b.shape
    
    b_old = b.copy()
    secs = example_MKL.solve_LAPACKE(nrep,n,nrh,A,b,threads_internal,threads_omp)
    x[:] = b.copy()
    b[:] = b_old
    
    return x,secs

In [14]:
def solve_Eigen(A,b,x,threads_internal,threads_omp):
    
    nrep,n,_ = A.shape
    _,_,nrh = b.shape
    
    b_old = b.copy()
    secs = example_MKL.solve_Eigen(nrep,n,nrh,A,b,threads_internal,threads_omp)
    x[:] = b.copy()
    b[:] = b_old
    
    return x,secs

**Test solving multiple equations systems at once:**

In [15]:
# numpy
name = 'numpy'
A,b,x = allocate_solve_mat()
x,secs = solve_np(A,b)
print(f'{name:30s} {secs*1000:8.0f} ms')
x_np = x.copy()

# mkl
for (threads_internal,threads_omp) in threads_vec:

    name = f'mkl (internal={threads_internal:2d},omp={threads_omp:2d}):'
    A,b,x = allocate_solve_mat()
    x,secs = solve_LAPACKE(A,b,x,threads_internal,threads_omp)
    print(f'{name:30s} {secs*1000:8.0f} ms')

# check results are the same
max_abs_diff = np.max(np.abs(x_np-x))
assert max_abs_diff < 1e-7
    
# Eigen
for (threads_internal,threads_omp) in threads_vec:

    name = f'Eigen (internal={threads_internal:2d},omp={threads_omp:2d}):'
    A,b,x = allocate_solve_mat()
    x,secs = solve_Eigen(A,b,x,threads_internal,threads_omp)
    print(f'{name:30s} {secs*1000:8.0f} ms')     
    
# check results are the same
max_abs_diff = np.max(np.abs(x_np-x))
assert max_abs_diff < 1e-7    


numpy                              2808 ms


mkl (internal= 1,omp= 8):          2344 ms


mkl (internal= 2,omp= 4):          2451 ms


mkl (internal= 4,omp= 4):          2917 ms


Eigen (internal= 1,omp= 8):        2221 ms


Eigen (internal= 2,omp= 4):        3261 ms


Eigen (internal= 4,omp= 4):        3218 ms


## 6. <a id='toc6_'></a>[Sparse matrices](#toc0_)

In [16]:
nrep = 100

# A
url = 'https://math.nist.gov/pub/MatrixMarket2/SPARSKIT/tokamak/utm1700b.mtx.gz'
urllib.request.urlretrieve(url,'matrices/utm1700b.mtx.gz')
A = scipy.io.mmread('matrices/utm1700b.mtx.gz')
nrow,ncol = A.shape

# numpy
A_np = np.broadcast_to(A.toarray(),(nrep,nrow,ncol)) 

# csr
A_csr = scipy.sparse.csr_array(A)

nnz = A_csr.data.size
data = np.broadcast_to(A_csr.data,(nrep,nnz))
indices = np.broadcast_to(A_csr.indices,(nrep,nnz))
indptr = np.broadcast_to(A_csr.indptr,(nrep,ncol+1))

# b
url = 'https://math.nist.gov/pub/MatrixMarket2/SPARSKIT/tokamak/utm1700b_rhs1.mtx.gz'
urllib.request.urlretrieve(url,'matrices/utm1700b_rhs1.mtx.gz')
b = scipy.io.mmread('matrices/utm1700b_rhs1.mtx.gz')
b = np.broadcast_to(b.ravel(),(nrep,nrow))


**Test solving multiple equations systems at once:**

In [17]:
# numpy
name = 'numpy' 
x_np = np.zeros((nrep,nrow))

t0 = time.perf_counter()
for i in range(nrep):
    x_np[i] = np.linalg.solve(A_np[i],b[i])
secs = time.perf_counter()-t0
print(f'{name:30s}: {secs*1000:8.0f} ms')

for i in range(nrep): assert np.max(np.abs(A_np[i]@x_np[i]-b[i])) < 1e-7

# spsolve
name = 'spsolve' 
x_sp = np.zeros((nrep,nrow))

t0 = time.perf_counter()
for i in range(nrep):
    x_sp[i] = scipy.sparse.linalg.spsolve(A_csr,b[i])
secs = time.perf_counter()-t0
print(f'{name:30s}: {secs*1000:8.0f} ms')

for i in range(nrep): assert np.max(np.abs(A_csr@x_sp[i]-b[i])) < 1e-7

# Eigen
name = 'Eigen_ParadisoLU'
x_Eigen = np.zeros((nrep,nrow))

for (threads_internal,threads_omp) in threads_vec:
    
    name = f'Eigen (internal={threads_internal:2d},omp={threads_omp:2d}):'
    secs = example_MKL.solve_sparse_Eigen(nrep,nrow,ncol,nnz,indptr,indices,data,b,x_Eigen,
                                          threads_internal,threads_omp)
    print(f'{name:30s}: {secs*1000:8.0f} ms')

for i in range(nrep): assert np.max(np.abs(A_csr@x_Eigen[i]-b[i])) < 1e-7

numpy                         :     9013 ms


spsolve                       :     1268 ms


Eigen (internal= 1,omp= 8):   :      867 ms


Eigen (internal= 2,omp= 4):   :     1024 ms


Eigen (internal= 4,omp= 4):   :     1227 ms


## 7. <a id='toc7_'></a>[Clean up](#toc0_)

In [18]:
example_MKL.clean_up()

In [19]:
os.remove(f'cppfuncs/Eigen-main.zip')
shutil.rmtree(f'cppfuncs/Eigen/')